# This is an attempt to build GPT from scratch

In [1]:
# We always start with a dataset to train on - this is the shake speare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-06 00:00:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  2.98MB/s    in 0.4s    

2025-10-06 00:00:09 (2.98 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text= f.read()

In [3]:
chars= sorted(list(set(text)))
vocab_size= len(chars)
print(''.join(chars))
print(f"The size of the chractacter vocabulary is {vocab_size}, just note: there is also a space")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
The size of the chractacter vocabulary is 65, just note: there is also a space


### Cell Below: Is creating a simple tokenizer / way of encoding - very simple

Can look into other tokenizer packages like
- sentencepiece
- tiktokenizer

In [4]:
# Create a mapping from characters to integers and vice versa
stoi= {ch:i for i,ch in enumerate(chars)}
itos= {i:ch for i,ch in enumerate(chars)}

# these are functions

# encode a string into a list of integers
encode= lambda s: [stoi[c] for c in s]

# decode a list of integers into a string
decode= lambda l: ''.join([itos[i] for i in l])

print(encode("this is encoded"))
print(decode(encode("this is encoded")))

[58, 46, 47, 57, 1, 47, 57, 1, 43, 52, 41, 53, 42, 43, 42]
this is encoded


In [5]:
import torch

## this part as I'm understanding should really be moved towards the end
# the move to device should happen after everything can work on cpu first
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

torch.manual_seed(2147483647)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(2147483647)

Using device: cuda


In [6]:
## Encode the entire dataset into a tensor

data= torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
## Now to split up the data into train and validation sets
# This won't be shuffled it's just raw from the shakespeare texts
n= int(0.9*len(data))
train_data= data[:n]
val_data= data[n:]

In [8]:
block_size= 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

With {block_size}, we use a specific up to an amout of context in order to predict the next character/token
- This is the use of context from as little as 1 to {block_size} to inference

__The next few Cells are to demonstrate how we can use from 1 - {block_size} of context to predict the next token__

In [9]:
# an example to showcase how we can use up to 8 tokens of context
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [10]:
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

## This is literally just getting a batch or sample based on the above batch_size and block_size
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # Get random chunks of data / offsets within the split data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(f"this ix is of shape {ix.shape}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[43, 11,  0, 35, 46, 53,  6,  1],
        [50, 63,  1, 42, 43, 60, 53, 58],
        [ 1, 54, 47, 58, 63,  1, 57, 47],
        [ 1, 61, 47, 50, 50,  1, 52, 53]])
targets:
torch.Size([4, 8])
tensor([[11,  0, 35, 46, 53,  6,  1, 39],
        [63,  1, 42, 43, 60, 53, 58, 47],
        [54, 47, 58, 63,  1, 57, 47, 58],
        [61, 47, 50, 50,  1, 52, 53, 58]])
----
when input is [43] the target: 11
when input is [43, 11] the target: 0
when input is [43, 11, 0] the target: 35
when input is [43, 11, 0, 35] the target: 46
when input is [43, 11, 0, 35, 46] the target: 53
when input is [43, 11, 0, 35, 46, 53] the target: 6
when input is [43, 11, 0, 35, 46, 53, 6] the target: 1
when input is [43, 11, 0, 35, 46, 53, 6, 1] the target: 39
when input is [50] the target: 63
when input is [50, 63] the target: 1
when input is [50, 63, 1] the target: 42
when input is [50, 63, 1, 42] the target: 43
when input is [50, 63, 1, 42, 43] the target: 60
when input is [50, 63, 1

__Implementing Bigram Module__

In [11]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(1337)

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [12]:
## when defining a module, there are 3 things that must always be defined, init, forward, and generate
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        # what logits does: find the scores for each example for each input context to find all possibilities of scores
        #
        # The input tensor (idx) acts as a map of indices into the embedding table.
        # For each token in each sequence of the batch, we look up its corresponding row in the embedding table.
        # Each row contains 65 logits — one for every possible next token in the vocabulary.
        # The result is a (batch_size, block_size, vocab_size) tensor,
        # where each position in the input has a full probability distribution (logits) over the next possible token.
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        

m= BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.2938, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
# We saw that it was pretty random. we will now train the bigram model to do much better
optimizer= torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000):  # repeat training many times to optimize model parameters

    # ---- 1. Sample a mini-batch of data ----
    # get_batch('train') randomly selects 'batch_size' examples (x,y) pairs from the training data.
    # Each example consists of:
    #   xb: input tokens  (shape: [batch_size, block_size])
    #   yb: target tokens (shape: [batch_size, block_size])
    # So each token in xb corresponds to one "predict the next token" target in yb.
    xb, yb = get_batch('train')

    # ---- 2. Forward pass ----
    # Pass the input tokens through the model (BigramLanguageModel).
    # This computes:
    #   logits: model predictions (raw scores before softmax)
    #   loss: how wrong the predictions are, computed via cross-entropy
    logits, loss = m(xb, yb)

    # ---- 3. Reset gradients ----
    # Gradients from the previous iteration remain stored in each parameter’s .grad attribute.
    # We must clear them before computing new gradients; otherwise they'd accumulate.
    # 'set_to_none=True' is a slightly faster and more memory-efficient way to reset grads to None.
    optimizer.zero_grad(set_to_none=True)

    # ---- 4. Backward pass (gradient computation) ----
    # This is where PyTorch’s autograd system kicks in.
    # loss.backward() traverses the computation graph built during the forward pass
    # and computes the derivative (∂loss/∂param) for every learnable parameter.
    # The results are stored in param.grad for each parameter in the model.
    loss.backward()
    # this loss.backward() is possible because of the fact that nn.Module utilizes nn.Parameter which default sets
    # the requires_grad=True resulting in the computational graph to be constructed and the loss calculation to be possible

    # ---- 5. Parameter update ----
    # The optimizer (e.g., Adam or SGD) updates each parameter using its gradient.
    # The general rule: new_param = old_param - learning_rate * param.grad
    # This step moves the parameters slightly in the direction that reduces the loss.
    optimizer.step()

# ---- 6. Report final training loss ----
# After the loop ends, we print the final scalar loss value.
# Lower loss means the model’s predicted next-token probabilities are closer to the true data distribution.
print(loss.item())


2.382369041442871


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


# __Now we are going into SELF ATTENTION__

In [26]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(1337)
# we need to average the previous context
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [27]:
# want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [28]:
# The above isn't right
### There's a better way to trick into self attention with matrix multipllication
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(1337)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
--
c=
tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])


In [29]:
 # using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)
# xbow[0], xbow2[0]
# xbow[0] == xbow2[0]
# They are identiical not sure why they arne't showing true

False

In [32]:
# use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)
xbow[0], xbow3[0]
# they are identical !!

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [23]:
# self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [24]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [33]:
# ===============================================
# Educational walkthrough of SELF-ATTENTION
# ===============================================

# B = batch size (how many independent sequences processed in parallel)
# T = time steps / tokens in each sequence
# C = channels / embedding dimension (features per token)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)  # random input embeddings (like token embeddings for 8 tokens per sequence)

# ---------------------------------------------------------------
# Step 1. Define the size of one "attention head"
# ---------------------------------------------------------------
# head_size determines how many features this head will project into.
# Smaller than C because we’ll later have multiple heads that split the work.
head_size = 16

# ---------------------------------------------------------------
# Step 2. Create linear projections for Q (query), K (key), V (value)
# ---------------------------------------------------------------
# Each token's embedding vector (size C) will be projected into 3 separate spaces:
# - Query: what information this token wants to gather from others.
# - Key: what information this token offers to others.
# - Value: the actual content this token carries that might be shared.
# These are learnable linear layers — weights are trained during backprop.
key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

# ---------------------------------------------------------------
# Step 3. Compute Q, K, V for each token in the sequence
# ---------------------------------------------------------------
# After projection:
#  - q, k, v each have shape (B, T, head_size)
# Each token now has its own "query", "key", and "value" representation.
k = key(x)
q = query(x)
v = value(x)

# ---------------------------------------------------------------
# Step 4. Compute raw attention scores ("affinities")
# ---------------------------------------------------------------
# q @ k^T compares each token's query to every other token's key.
# This gives a matrix of pairwise similarities: how much should token i attend to token j?
# Shape math:
#   q: (B, T, head_size)
#   k^T: (B, head_size, T)
#   q @ k^T: (B, T, T)
# So for each sequence in the batch, we get a T x T attention map.
wei = q @ k.transpose(-2, -1)  # (B, T, T)

# ---------------------------------------------------------------
# Step 5. Apply causal mask (tril) to prevent "looking into the future"
# ---------------------------------------------------------------
# We create a lower-triangular matrix of ones.
# tril[i, j] = 1 means token i can attend to token j (j <= i).
# tril[i, j] = 0 means token i cannot attend to token j (j > i, the future).
tril = torch.tril(torch.ones(T, T))

# masked_fill replaces all forbidden future positions with -inf
# so that after softmax, those weights become zero (no attention).
wei = wei.masked_fill(tril == 0, float('-inf'))

# ---------------------------------------------------------------
# Step 6. Softmax to turn scores into attention weights
# ---------------------------------------------------------------
# Softmax normalizes each row of the (T x T) attention matrix into probabilities.
# Each row i now sums to 1 and represents: "how much attention token i gives to each past token".
wei = F.softmax(wei, dim=-1)  # (B, T, T), rows are probability distributions

# ---------------------------------------------------------------
# Step 7. Weight the value vectors using the attention weights
# ---------------------------------------------------------------
# Each token’s new representation is a weighted sum of all other tokens’ values.
# - wei: (B, T, T)
# - v:   (B, T, head_size)
# Result:
#   out[b, t, :] = sum_j (wei[b, t, j] * v[b, j, :])
# So each token now "blends" information from the tokens it attends to.
out = wei @ v  # (B, T, head_size)

# ---------------------------------------------------------------
# Step 8. Verify output shape
# ---------------------------------------------------------------
# out has the same sequence length (T) but new feature dimension (head_size).
# Each token’s output is now a context-aware vector — it knows which other tokens matter.
print(out.shape)  # (B, T, head_size)


torch.Size([4, 8, 16])

scaling is also used to control variance at initalization. attend to attention is all you need to learn more about it
Q @ K.T / (headsize)**-0.5

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5059
step 300: train loss 2.4198, val loss 2.4338
step 400: train loss 2.3503, val loss 2.3566
step 500: train loss 2.2963, val loss 2.3126
step 600: train loss 2.2411, val loss 2.2502
step 700: train loss 2.2053, val loss 2.2189
step 800: train loss 2.1629, val loss 2.1865
step 900: train loss 2.1244, val loss 2.1509
step 1000: train loss 2.1022, val loss 2.1288
step 1100: train loss 2.0705, val loss 2.1196
step 1200: train loss 2.0392, val loss 2.0800
step 1300: train loss 2.0255, val loss 2.0647
step 1400: train loss 1.9935, val loss 2.0377
step 1500: train loss 1.9697, val loss 2.0294
step 1600: train loss 1.9611, val loss 2.0456
step 1700: train loss 1.9415, val loss 2.0134
step 1800: train loss 1.9074, val loss 1.9941
step 1900: train loss 1.9087, val loss 1.9884
step 2000: train loss 1.8858, val loss 1.9976
step 2100: train loss 1.